In [10]:
import cv2
import numpy as np
import os
import time
import mediapipe as mp

# ⚙️ CONFIG
DATA_DIR = "sign_data"
label = input("Enter label (e.g., 'hello', 'pause', 'thank'): ").strip()
label_dir = os.path.join(DATA_DIR, label)
os.makedirs(label_dir, exist_ok=True)

max_sequences = 60
frames_per_seq = 30

# MediaPipe
mp_holistic = mp.solutions.holistic
mp_draw = mp.solutions.drawing_utils
holistic = mp_holistic.Holistic()

cap = cv2.VideoCapture(0)

sequence = []
seq_count = 0
recording = False
countdown = False
countdown_start = 0

print("กด 's' เพื่อเริ่มเก็บทั้งหมด | กด 'q' เพื่อออก")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = holistic.process(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # วาด Landmark
    if results.left_hand_landmarks:
        mp_draw.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    if results.right_hand_landmarks:
        mp_draw.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    if results.pose_landmarks:
        mp_draw.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    if results.face_landmarks:
        mp_draw.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION)

    # ดึง keypoints
    keypoints = []
    def extract(lms, count):
        return [[lm.x, lm.y, lm.z] for lm in lms.landmark] if lms else [[0, 0, 0]] * count

    keypoints += extract(results.face_landmarks, 468)
    keypoints += extract(results.left_hand_landmarks, 21)
    keypoints += extract(results.right_hand_landmarks, 21)
    keypoints += extract(results.pose_landmarks, 33)

    keypoints = np.array(keypoints).flatten()

    # Countdown once before recording starts
    if countdown:
        elapsed = time.time() - countdown_start
        remaining = int(3 - elapsed)
        if remaining > 0:
            cv2.putText(image, f"Starting in {remaining}s", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        else:
            countdown = False
            recording = True

    # Recording mode: auto collect all sequences
    if recording:
        sequence.append(keypoints)
        cv2.putText(image, f"Recording... Frame {len(sequence)}/30", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        if len(sequence) == frames_per_seq:
            save_path = os.path.join(label_dir, f"{seq_count}.npy")
            np.save(save_path, sequence)
            print(f"✅ Saved: {save_path}")
            seq_count += 1
            sequence = []
        if seq_count == max_sequences:
            print("✅ เก็บครบแล้ว"); recording = False

    # UI
    cv2.putText(image, f"Label: {label} | Sequence: {seq_count}/{max_sequences}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
    cv2.imshow("Dataset Collector", image)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('s') and not countdown and not recording and seq_count < max_sequences:
        countdown = True
        countdown_start = time.time()

    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Enter label (e.g., 'hello', 'pause', 'thank'):  tea


กด 's' เพื่อเริ่มเก็บทั้งหมด | กด 'q' เพื่อออก
✅ Saved: sign_data\tea\0.npy
✅ Saved: sign_data\tea\1.npy
✅ Saved: sign_data\tea\2.npy
✅ Saved: sign_data\tea\3.npy
✅ Saved: sign_data\tea\4.npy
✅ Saved: sign_data\tea\5.npy
✅ Saved: sign_data\tea\6.npy
✅ Saved: sign_data\tea\7.npy
✅ Saved: sign_data\tea\8.npy
✅ Saved: sign_data\tea\9.npy
✅ Saved: sign_data\tea\10.npy
✅ Saved: sign_data\tea\11.npy
✅ Saved: sign_data\tea\12.npy
✅ Saved: sign_data\tea\13.npy
✅ Saved: sign_data\tea\14.npy
✅ Saved: sign_data\tea\15.npy
✅ Saved: sign_data\tea\16.npy
✅ Saved: sign_data\tea\17.npy
✅ Saved: sign_data\tea\18.npy
✅ Saved: sign_data\tea\19.npy
✅ Saved: sign_data\tea\20.npy
✅ Saved: sign_data\tea\21.npy
✅ Saved: sign_data\tea\22.npy
✅ Saved: sign_data\tea\23.npy
✅ Saved: sign_data\tea\24.npy
✅ Saved: sign_data\tea\25.npy
✅ Saved: sign_data\tea\26.npy
✅ Saved: sign_data\tea\27.npy
✅ Saved: sign_data\tea\28.npy
✅ Saved: sign_data\tea\29.npy
✅ Saved: sign_data\tea\30.npy
✅ Saved: sign_data\tea\31.npy
✅ S

In [12]:
import os
import numpy as np
from sklearn.model_selection import train_test_split

# -----------------------------
# 🔧 CONFIG
DATA_DIR = "sign_data"  # ตรงกับที่พี่เก็บไว้
LABELS = sorted(os.listdir(DATA_DIR))  # ['hello', 'pause', ...]
label_map = {label: idx for idx, label in enumerate(LABELS)}
print("📌 label_map:", label_map)

X, y = [], []

# -----------------------------
# 📥 LOAD DATA
for label in LABELS:
    folder_path = os.path.join(DATA_DIR, label)
    files = os.listdir(folder_path)
    for file in files:
        data = np.load(os.path.join(folder_path, file))  # shape (30, 1629)
        if data.shape == (30, 1629):  # ป้องกันข้อมูลเสีย
            X.append(data)
            y.append(label_map[label])
        else:
            print(f"⚠️ Skipped: {file} (shape = {data.shape})")

X = np.array(X)  # shape = (N, 30, 1629)
y = np.array(y)
print(f"\n✅ Loaded {X.shape[0]} sequences, shape = {X.shape}, labels = {len(np.unique(y))}")

# -----------------------------
# 🔀 SPLIT TRAIN / VAL (ถ้าใช้เทรนต่อ)
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

print("\n🎯 Train Shape:", X_train.shape)
print("🎯 Val Shape:", X_val.shape)


📌 label_map: {'I': 0, 'hello': 1, 'need': 2, 'pause': 3, 'tea': 4}

✅ Loaded 300 sequences, shape = (300, 30, 1629), labels = 5

🎯 Train Shape: (240, 30, 1629)
🎯 Val Shape: (60, 30, 1629)


In [14]:
np.save("X_train.npy", X_train)
np.save("y_train.npy", y_train)
np.save("X_val.npy", X_val)
np.save("y_val.npy", y_val)

print("✅ Saved all dataset files successfully!")


✅ Saved all dataset files successfully!


In [16]:
import numpy as np

X_train = np.load("X_train.npy")
y_train = np.load("y_train.npy")
X_val = np.load("X_val.npy")
y_val = np.load("y_val.npy")

print("✅ ข้อมูลโหลดสำเร็จ:", X_train.shape, y_train.shape)


✅ ข้อมูลโหลดสำเร็จ: (240, 30, 1629) (240,)


In [18]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

# 🔧 Model Architecture
model = Sequential([
    Input(shape=(30, 1629)),
    LSTM(64, return_sequences=True),
    Dropout(0.3),
    LSTM(32),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(len(np.unique(y_train)), activation='softmax')  # auto match 5 classes
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# ⏱️ Callback
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# 🚀 Train
history = model.fit(X_train, y_train, 
                    epochs=50, 
                    validation_data=(X_val, y_val),
                    callbacks=[early_stop],
                    batch_size=16)

# 💾 Save
model.save("sign_language_model.keras")
print("✅ โมเดลเทรนเสร็จและบันทึกเรียบร้อย")


Epoch 1/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - accuracy: 0.1177 - loss: 1.6469 - val_accuracy: 0.2000 - val_loss: 1.6103
Epoch 2/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.2221 - loss: 1.6087 - val_accuracy: 0.2000 - val_loss: 1.6076
Epoch 3/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.2114 - loss: 1.6173 - val_accuracy: 0.2000 - val_loss: 1.6039
Epoch 4/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.1679 - loss: 1.6152 - val_accuracy: 0.2000 - val_loss: 1.6036
Epoch 5/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.2124 - loss: 1.6161 - val_accuracy: 0.4000 - val_loss: 1.5971
Epoch 6/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.2276 - loss: 1.6001 - val_accuracy: 0.3500 - val_loss: 1.5827
Epoch 7/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.2332 - loss: 1.5940 - val_accuracy: 0.4000 - val_loss: 1.5529
Epoch 8/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.4155 - loss: 1.5293 - val_accuracy: 0.5500 - v

In [2]:
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.title("Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title("Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

plt.tight_layout()
plt.show()


NameError: name 'plt' is not defined

In [ ]:
import sys
import cv2
import numpy as np
import tensorflow as tf
import mediapipe as mp
from PyQt5.QtWidgets import QApplication, QLabel, QWidget, QVBoxLayout
from PyQt5.QtGui import QImage, QPixmap
from PyQt5.QtCore import QTimer

# โหลดโมเดลและเตรียม label
model = tf.keras.models.load_model("sign_language_model.keras")
LABELS = ['hello', 'I', 'need', 'pause', 'tea']  # แก้ให้ตรงกับโฟลเดอร์ที่พี่ใช้เทรน
label_map = {i: label for i, label in enumerate(LABELS)}

mp_holistic = mp.solutions.holistic
holistic = mp_holistic.Holistic()

class SignApp(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Sign Language Detection with Pause")
        self.setGeometry(100, 100, 800, 600)

        self.image_label = QLabel(self)
        self.status_label = QLabel("Status: Waiting for sign")
        self.sentence_label = QLabel("")
        self.status_label.setStyleSheet("font-size: 18px; color: blue")
        self.sentence_label.setStyleSheet("font-size: 22px; color: green")

        layout = QVBoxLayout()
        layout.addWidget(self.image_label)
        layout.addWidget(self.status_label)
        layout.addWidget(self.sentence_label)
        self.setLayout(layout)

        self.cap = cv2.VideoCapture(0)
        self.timer = QTimer()
        self.timer.timeout.connect(self.update_frame)
        self.timer.start(30)

        self.sequence = []
        self.sentence = []
        self.waiting_for_pause = False
        self.last_label = None

    def update_frame(self):
        ret, frame = self.cap.read()
        if not ret:
            return

        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = holistic.process(rgb)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # แก้สีฟ้าให้ปกติ

        keypoints = []
        def extract(lms, count):
            return [[lm.x, lm.y, lm.z] for lm in lms.landmark] if lms else [[0, 0, 0]] * count

        keypoints += extract(results.face_landmarks, 468)
        keypoints += extract(results.left_hand_landmarks, 21)
        keypoints += extract(results.right_hand_landmarks, 21)
        keypoints += extract(results.pose_landmarks, 33)
        keypoints = np.array(keypoints).flatten()

        # เก็บ frame ลง sequence
        self.sequence.append(keypoints)
        if len(self.sequence) > 30:
            self.sequence.pop(0)

        if len(self.sequence) == 30:
            input_data = np.array(self.sequence).reshape(1, 30, 1629)
            prediction = model.predict(input_data, verbose=0)
            confidence = np.max(prediction)
            label = label_map[np.argmax(prediction)]

            if confidence > 0.85:
                if not self.waiting_for_pause:
                    if label != "pause" and label != self.last_label:
                        self.sentence.append(label)
                        self.waiting_for_pause = True
                        self.status_label.setText(f"Detected: {label} → Waiting for pause...")
                        self.last_label = label
                else:
                    if label == "pause":
                        self.sentence.append("|")
                        self.waiting_for_pause = False
                        self.status_label.setText("✅ Pause detected. Ready for next sign.")
                        self.last_label = None

        self.sentence_label.setText(" ".join([s for s in self.sentence if s != "|"]))

        h, w, ch = image.shape
        bytes_per_line = ch * w
        q_img = QImage(image.data, w, h, bytes_per_line, QImage.Format_RGB888)
        self.image_label.setPixmap(QPixmap.fromImage(q_img))

    def closeEvent(self, event):
        self.cap.release()
        cv2.destroyAllWindows()

if __name__ == '__main__':
    app = QApplication(sys.argv)
    window = SignApp()
    window.show()
    sys.exit(app.exec_())
